In [227]:
import numpy as np
import pandas as pd
import plotly.express as px

def estimate_loads_from_strains_with_errors_per_gauge(
    measured_time_vs_strain_data_file_path, 
    sensitivity_matrix_file_path,
    signal_noise_microstrains=20,  # Signal noise in microstrains
    gage_factor_error_percent=1,   # Gage factor error in percent
    positioning_error_percent=3    # Positioning error in percent
):
    # Load the CSV files
    S_df = pd.read_csv(measured_time_vs_strain_data_file_path)
    A_df = pd.read_csv(sensitivity_matrix_file_path, header=None)

    # Extract strain measurements and sensitivity matrix
    S_matrix = S_df.iloc[:, 1:].values
    A_matrix = A_df.values

    # Calculate variances from different error sources for each gauge
    signal_noise = signal_noise_microstrains * 1e-6
    signal_noise_variance = signal_noise ** 2

    # Calculate RMS strain for each gauge
    rms_strain_per_gauge = np.sqrt(np.mean(S_matrix**2, axis=0))

    gage_factor_error = gage_factor_error_percent / 100
    positioning_error = positioning_error_percent / 100

    # Total variance for each strain gauge measurement
    total_variance_per_gauge = (signal_noise_variance +
                                (gage_factor_error ** 2) * rms_strain_per_gauge ** 2 +
                                (positioning_error ** 2) * rms_strain_per_gauge ** 2)
    
    total_variance_per_gauge[total_variance_per_gauge == 0] = 1e-10  # replace 0 with a small number

    # Creating the W matrix for each gauge
    W = np.diag(1 / total_variance_per_gauge)

    # Perform the weighted least squares estimate calculation
    A_T_W = A_matrix.T @ W
    A_T_W_A_inv = np.linalg.inv(A_T_W @ A_matrix)
    L_hat = A_T_W_A_inv @ A_T_W @ S_matrix.T

    # Transpose to get the original shape
    L_hat_timeseries = L_hat.T

    # Create a DataFrame for the results
    estimated_loads_df = pd.DataFrame(L_hat_timeseries, columns=[f'Load {i+1}' for i in range(A_matrix.shape[1])])
    estimated_loads_df.insert(0, 'Time [s]', S_df.iloc[:, 0])

    # Save to CSV
    output_csv_file_path = 'estimated_loads_with_errors_per_gauge_RMS.csv'
    estimated_loads_df.to_csv(output_csv_file_path, index=False)
    estimated_loads_df.set_index('Time [s]', inplace=True)

    return estimated_loads_df, output_csv_file_path

# Example usage of the function
loads_df_with_errors_per_gauge, csv_file_path_with_errors_per_gauge = estimate_loads_from_strains_with_errors_per_gauge(
    'sample_input_csv_1.csv',  # Your CSV file paths
    'strain_sensitivity_matrix.csv',
    0, 0, 0  # Error parameters
)

print(loads_df_with_errors_per_gauge)  # Display the first few rows of the estimated loads
print(f"CSV file saved at: {csv_file_path_with_errors_per_gauge}")

px.line(loads_df_with_errors_per_gauge)


                Load 1        Load 2        Load 3
Time [s]                                          
0.0125    1.567245e+02  1.117944e-01  2.599926e+04
0.0250    3.128784e+02  2.135032e-01  5.232680e+04
0.0375    4.669172e+02  3.134697e-01  7.825795e+04
0.0500    6.180712e+02  4.222155e-01  1.038943e+05
0.0625    7.650658e+02  4.985733e-01  1.296881e+05
...                ...           ...           ...
0.9500   -6.179889e+02 -4.182466e-01  1.048856e+05
0.9630   -4.662783e+02 -3.280222e-01  7.889037e+04
0.9750   -3.125260e+02 -2.255094e-01  5.288855e+04
0.9880   -1.567719e+02 -1.098185e-01  2.627570e+04
1.0000   -2.001318e-11 -1.414393e-14  3.358970e-09

[80 rows x 3 columns]
CSV file saved at: estimated_loads_with_errors_per_gauge_RMS.csv
